In [8]:
import pandas as pd
import glob
import os
import re
import datetime

#폴더 내 csv 파일 찾기
folder = input("데이터가 있는 폴더:")
csv_files = glob.glob(os.path.join(folder, "*.csv"))
for file in csv_files:
    print(f"{os.path.basename(file)}")
print(f"총{len(csv_files)}개의 csv 파일을 찾았습니다\n")

# 파일명을 통해 최근 순으로 파일 정리. 
most_recent_file = None
max_year=0
year_pattern = re.compile(r'\d{4}') #숫자 4개가 연속으로 이어질 때 파싱
files_with_year=[]
for file in csv_files:
    file_name = os.path.basename(file)
    match = year_pattern.search(file_name)
    if match:
        year= int(match.group())
        files_with_year.append((year, file))
files_with_year.sort(key=lambda x:x[0], reverse=True)

#데이타의 칼럼 수와 샘플 수 확인
first_year= True
for year, file in files_with_year:
    try:
        encoding ="cp949"
        df = pd.read_csv(file, encoding=encoding, nrows=0)
    except UnicodeDecodeError:
        encoding="utf-8"
        df= pd.read_csv(file, encoding=encoding, nrows=0)
    with open(file, "r", encoding=encoding) as f:
        line_count=0
        for line in f:
            line_count +=1
        print(f"{year}년: {line_count:,}개의 샘플, {len(df.columns)}개의 칼럼.")
    if first_year :
        col_to_read=df.columns
        first_year = False


# 몇 개나 합할 것인지?
df_list=[]
num_of_combined=int(input("병합할 파일 갯수:"))
if num_of_combined <= 0 : 
    import sys
    sys.exit()

for i in range(num_of_combined):
    print(f"{files_with_year[i][1]}파일을 읽습니다.")
    df= pd.read_csv(files_with_year[i][1], encoding=encoding, usecols= col_to_read)
    df=df.reindex(columns=col_to_read)
    df= df[1:] #0행에 칼럼 영문 코드가 있는 경우가 종종 있음.
    df_list.append(df)

df=pd.concat(df_list, ignore_index=True) 
del df_list   
   
#dType이 object인 열을 레이블인지 결측인지 판단
col_object= df.select_dtypes(include=['object']).columns.tolist()
col_code_keyword =['코드','여부']
col_code=[]
for col in col_object:
    if any(keyword in col for keyword in col_code_keyword):
        col_code.append(col)
col_error = [col for col in col_object if col not in col_code]
print(f"{len(col_code)}개의 문자형 범주 칼럼.\n 데이타가 고르지 못한 칼럼은 {col_error}.")

#원본 칼럼명에 원래 레이블과 숫자로 변환된 레이블을 튜플로 저장
original_col_code_list=[]
from sklearn.preprocessing import LabelEncoder
for col in col_code:
    le=LabelEncoder()
    df[col]=le.fit_transform(df[col])
    original_col_code_list.append((col, dict(zip(le.classes_,le.transform(le.classes_)))))
    print(f"{col}의 코드를 숫자로 변환")

#결측 데이타 처리
for col in col_error:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    print(f"{col} 칼럼의 데이터 에러를 수정")
nan_num = df.isnull().any(axis=1).sum()
origin_num=len(df)
print(f"(NaN 행의 수는 {nan_num:,}개. 전체 대비 {nan_num/origin_num:.1%}.")
del_rows = input("NaN 행들을 지우시겠습니까?(y/n)")
if del_rows == 'y':
    df= df.dropna()
    print(f"{(origin_num-len(df)):,}가 삭제.")

#데이타 레이블 맵핑 및 설명
file_path_excel=input("항목정보가 있는 파일명(엑셀):")
df_feature_info=pd.read_excel(os.path.join(folder, file_path_excel), sheet_name="총괄", header=2,)
col_to_read_exel=['항목명', '코드','코드의미 및 단위' ]
df_feature_info=df_feature_info[col_to_read_exel]                              

for i in range(len(original_col_code_list)):
    condition = df_feature_info['항목명'] ==original_col_code_list[i][0]
    df_feature_info.loc[condition, '코드']=df_feature_info.loc[condition, '코드'].map(original_col_code_list[0][1])

df_feature_info.to_csv(os.path.join(folder, "항목정보.csv"), encoding="cp949")
print("항목정보.csv 파일을 생성.")

saveornot= input(f"모든 처리가 종료. {len(df)}개 Data를 저장?(y/n)")
if saveornot=="y":
    now=datetime.datetime.now()
    now_fomatted=now.strftime("%m-%d_%H-%M")
    df_filename= f"고용형태별조사DB_{now_fomatted}.csv"
    print("파일 저장 중...............")
    df.to_csv(os.path.join(folder,df_filename), encoding="cp949")
    print(f"{df_filename}을 저장.")
    del df



                            

데이터가 있는 폴더: data2


2021_총괄_20250827_24235.csv
2022_총괄_20250827_24235.csv
2023_총괄_20250827_62500.csv
2024_총괄_20250827_62500.csv
고용형태별조사DB_09-02_15-31.csv
항목정보.csv
총6개의 csv 파일을 찾았습니다

2024년: 1,004,559개의 샘플, 29개의 칼럼.
2023년: 991,358개의 샘플, 29개의 칼럼.
2022년: 993,588개의 샘플, 29개의 칼럼.
2021년: 969,620개의 샘플, 29개의 칼럼.


병합할 파일 갯수: 1


data2\2024_총괄_20250827_62500.csv파일을 읽습니다.


C:\Users\jwhan\AppData\Local\Temp\ipykernel_25828\903797535.py:55: DtypeWarning: Columns (11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(files_with_year[i][1], encoding=encoding, usecols= col_to_read)


...

25,153가 삭제.


항목정보가 있는 파일명(엑셀): 2024년_고용형태별근로실태조사_파일설계서.xlsx


항목정보.csv 파일을 생성.


모든 처리가 종료. 979404개 Data를 저장?(y/n) y


고용형태별조사DB_09-02_15-42.csv을 저장.


In [3]:
col_code

['산업대분류코드']

In [4]:
col_object

['산업대분류코드', '소정실근로일수', '휴일실근로일수', '소정실근로시간수', '초과실근로시간수', '휴일실근로시간수']

In [5]:
df

NameError: name 'df' is not defined

In [6]:
import psutil
import os

# 현재 파이썬 프로세스 ID 가져오기
pid = os.getpid()

# PID를 이용해 프로세스 객체 생성
process = psutil.Process(pid)

# 메모리 정보 가져오기 (bytes 단위)
mem_info = process.memory_info()

# RSS(Resident Set Size) 메모리 확인
# RSS는 물리적(RAM) 메모리에 올라와 있는 실제 메모리 사용량입니다.
print(f"현재 파이썬 프로세스가 사용하는 메모리: {mem_info.rss / (1024 * 1024):.2f} MB")

현재 파이썬 프로세스가 사용하는 메모리: 207.89 MB
